In [ ]:
!pip install jschon

In [2]:
from jschon import create_catalog,JSON, JSONSchema
import json,os
import pathlib
import inspect



In [3]:
create_catalog('2020-12')


Catalog('catalog')

In [4]:
os.path.dirname(os.path.abspath("__file__"))


'd:\\Yanis\\FAC\\M1\\S2\\PSTL\\experiments\\validation'

In [4]:
src_file_path = os.path.dirname(os.path.abspath("__file__"))

# print(src_file_path)
dataset_dir =  src_file_path+'/dataset/1'
schema_dir = f'{dataset_dir}/schema.json'
instance_dir=f'{dataset_dir}/instance.json'

In [55]:
keywords_to_avoid = ['','anyOf','oneOf','allOf','patternPropreties']
def extract_errors_map(error:dict, error_map:dict):
    if 'keywordLocation' in error:
        key = error['keywordLocation'].split('/')[-1]
        if key and key not in keywords_to_avoid:
            if error_map.get(key) is None : 
                error_map[key]=[error['error']]
            else : 
                error_map[key].append(error['error'])
    
    # print("there's still error")
    if 'errors' in error:
        for e in error['errors']:
            extract_errors_map(e, error_map)

def extract_errors(errors): 
    error_map= {}
    for error in errors : 
        extract_errors_map(error,error_map)
    return list(error_map.keys())

In [63]:
def get_errors_validation(schema_dir:str,instance_dir:str): 
    
    with open(schema_dir,'r',encoding='utf-8') as f : 
        schema= json.load(f)
    schema['$schema'] ="https://json-schema.org/draft/2020-12/schema"
    schema['$id'] ="https://jschon.dev/demo"
    demo_schema = JSONSchema(schema)
    # print(demo_schema)
    with open (instance_dir,'r',encoding='utf-8') as f : 
        instance=json.load(f)
        
    result = demo_schema.evaluate(
        JSON(instance)
    )
    # print(result.output('detailed'))# if errors in result.output('detailed')
    if(result.output("detailed")['valid']==False or "errors" in result.output("detailed")): 
        errors = extract_errors(result.output('detailed')['errors'])
    else : 
        errors=[]
    return errors

In [64]:

get_errors_validation(schema_dir,instance_dir)

['required', 'type']